# Imports

In [20]:
from datasets import load_dataset, load_from_disk
import seaborn as sns
import os
from huggingface_hub import notebook_login
from rich.pretty import pprint

print("Modlues imported sucessfully")

Modlues imported sucessfully


In [ ]:
notebook_login() # some of us needed it for downloading the dataset

print("Login successful")

Login successful


# Analysis of the dataset
## Dataset features
### Core Study Information
- **nct_id:** Unique study identifier (NCT########)  
- **brief_title:** Concise study title  
- **official_title:** Complete formal study title  
- **study_type:** Study design type (Interventional, Observational, etc.)  
- **phases:** Clinical trial phases (Phase I, II, III, IV)  
- **overall_status:** Current study status  

### Study Design & Population
- **enrollment_count:** Target or actual enrollment number  
- **minimum_age / maximum_age:** Age eligibility bounds  
- **sex:** Gender eligibility (All, Male, Female)  
- **healthy_volunteers:** Whether healthy volunteers are accepted  
- **eligibility_criteria:** Detailed inclusion/exclusion criteria  
- **study_population:** Target population description  

### Clinical Context
- **conditions:** Medical conditions studied  
- **keywords:** Study-related keywords  
- **brief_summary:** Study purpose and rationale  
- **detailed_description:** Comprehensive study description  
- **primary_outcomes / secondary_outcomes:** Measured endpoints  

### Organizational Information
- **lead_sponsor:** Primary study sponsor  
- **collaborators:** Additional supporting organizations  
- **locations:** Study sites with geographic coordinates  
- **overall_officials:** Principal investigators and study officials  

### Temporal Data
- **study_first_submit_date:** Initial submission to ClinicalTrials.gov  
- **start_date:** Study start date  
- **primary_completion_date:** Primary endpoint completion date  
- **completion_date:** Overall study completion date  
- **last_update_submit_date:** Most recent data update date  

### Enhanced Features
- **brief_summary_embedding:** 768-dimensional semantic embedding of study summary  
- **eligibility_criteria_embedding:** 768-dimensional embedding of eligibility text  

## Load dataset
if dataset is present on your device, then load it from disk, otherwise, load it from internet and save it on disk

In [3]:
path_to_ds = "clinical-dataset" # your path to the dataset

print(f"Path is set to: \"{path_to_ds}\"")

Path is set to: "clinical-dataset"


In [14]:

if os.path.exists(path_to_ds):
    dataset = load_from_disk(path_to_ds)
else:
    dataset = load_dataset("louisbrulenaudet/clinical-trials")
    dataset.save_to_disk(path_to_ds)

train_data = dataset["train"]

print("Dataset loaded successfully")


Dataset loaded successfully


### Print dataset details

In [22]:
print(f"Dataset size: {len(train_data)}")
print("Features:")
pprint(train_data.features)

Dataset size: 541897
Features:


{
│   'nct_id': Value('string'),
│   'updated_at': Value('timestamp[us]'),
│   'brief_title': Value('string'),
│   'official_title': Value('string'),
│   'acronym': Value('string'),
│   'study_type': Value('string'),
│   'overall_status': Value('string'),
│   'study_first_submit_date': Value('timestamp[ms]'),
│   'start_date': Value('timestamp[ms]'),
│   'primary_completion_date': Value('timestamp[ms]'),
│   'completion_date': Value('timestamp[ms]'),
│   'phases': List(Value('string')),
│   'enrollment_count': Value('float64'),
│   'minimum_age': Value('float64'),
│   'maximum_age': Value('float64'),
│   'sex': Value('string'),
│   'healthy_volunteers': Value('bool'),
│   'brief_summary': Value('string'),
│   'detailed_description': Value('string'),
│   'eligibility_criteria': Value('string'),
│   'lead_sponsor_name': Value('string'),
│   'lead_sponsor_class': Value('string'),
│   'org_study_id_info': {'id': Value('string'), 'link': Value('string'), 'type': Value('string')},
│   'why_stopped': Value('string'),
│   'expanded_access_info': {
│   │   'hasExpandedAccess': Value('bool'),
│   │   'nctId': Value('string'),
│   │   'statusForNctId': Value('string')
│   },
│   'last_update_submit_qc_date': Value('timestamp[ms]'),
│   'last_update_post_date_struct': {'date': Value('string'), 'type': Value('string')},
│   'study_first_post_date_struct': {'date': Value('string'), 'type': Value('string')},
│   'std_ages': List(Value('string')),
│   'study_population': Value('string'),
│   'sampling_method': Value('string'),
│   'oversight_has_dmc': Value('bool'),
│   'design_info': {
│   │   'allocation': Value('string'),
│   │   'interventionModel': Value('string'),
│   │   'interventionModelDescription': Value('string'),
│   │   'maskingInfo': {
│   │   │   'masking': Value('string'),
│   │   │   'maskingDescription': Value('string'),
│   │   │   'whoMasked': List(Value('string'))
│   │   },
│   │   'observationalModel': Value('string'),
│   │   'primaryPurpose': Value('string'),
│   │   'timePerspective': Value('string')
│   },
│   'conditions': List(Value('string')),
│   'keywords': Value('string'),
│   'interventions': Value('null'),
│   'locations': List({'city': Value('string'), 'country': Value('string'), 'facility': Value('string'), 'geoPoint': {'lat': Value('float64'), 'lon': Value('float64')}, 'state': Value('string')}),
│   'collaborators': List({'class': Value('string'), 'name': Value('string')}),
│   'arm_groups': Value('null'),
│   'outcomes': {
│   │   'other': List({'description': Value('null'), 'measure': Value('string'), 'timeFrame': Value('string')}),
│   │   'primary': List({'description': Value('null'), 'measure': Value('string'), 'timeFrame': Value('string')}),
│   │   'secondary': List({'description': Value('null'), 'measure': Value('string'), 'timeFrame': Value('string')})
│   },
│   'overall_officials': List({'affiliation': Value('string'), 'name': Value('string'), 'role': Value('string')}),
│   'study_references': Value('string'),
│   'misc_info_module': Value('string'),
│   'condition_browse_module': {
│   │   'ancestors': List({'id': Value('string'), 'term': Value('string')}),
│   │   'browseBranches': List({'abbrev': Value('string'), 'name': Value('string')}),
│   │   'browseLeaves': List({'asFound': Value('string'), 'id': Value('string'), 'name': Value('string'), 'relevance': Value('string')}),
│   │   'meshes': List({'id': Value('string'), 'term': Value('string')})
│   },
│   'intervention_browse_module': {
│   │   'ancestors': List({'id': Value('string'), 'term': Value('string')}),
│   │   'browseBranches': List({'abbrev': Value('string'), 'name': Value('string')}),
│   │   'browseLeaves': List({'asFound': Value('string'), 'id': Value('string'), 'name': Value('string'), 'relevance': Value('string')}),
│   │   'meshes': List({'id': Value('string'), 'term': Value('string')})
│   },
│   'mesh_terms': {
│   │   'conditions': List({'id': Value('string'), 'term': Value('string')}),
│   │   'interventions': List({

### Example of data

In [62]:
sample = dataset["train"].select(range(10)).to_pandas()

sample

# sample[["brief_title", "brief_summary", "detailed_description", "eligibility_criteria", "keywords"]] # Free text 
# sample[["mesh_terms", "condition_browse_module", "intervention_browse_module"]] # Structured text
# sample[["conditions", "interventions"]] # Validation data

,nct_id,updated_at,brief_title,official_title,acronym,study_type,overall_status,study_first_submit_date,start_date,primary_completion_date,...,locations,collaborators,arm_groups,outcomes,overall_officials,study_references,misc_info_module,condition_browse_module,intervention_browse_module,mesh_terms
0,NCT01273766,NaT,Deferasirox in Treating Iron Overload Caused B...,Impact of Intervention With Deferasirox on the...,None,INTERVENTIONAL,COMPLETED,2011-01-07,NaT,NaT,...,"[{'city': 'Winston-Salem', 'country': 'United ...","[{'class': 'NIH', 'name': 'National Cancer Ins...",None,"{'other': None, 'primary': [{'description': No...",[{'affiliation': 'Wake Forest University Healt...,None,"{""versionHolder"": ""2025-06-18""}","{'ancestors': [{'id': 'D004194', 'term': 'Dise...","{'ancestors': [{'id': 'D007502', 'term': 'Iron...","{'conditions': [{'id': 'D002051', 'term': 'Bur..."
1,NCT00787566,NaT,"Phase 2 Study of Efficacy, Tolerability, and S...","A Randomized, Single Administration, Double-bl...",None,INTERVENTIONAL,COMPLETED,2008-11-05,NaT,NaT,...,[{'city': 'The study is managed by Kendle Inte...,None,None,"{'other': None, 'primary': [{'description': No...",None,None,"{""versionHolder"": ""2025-06-18""}","{'ancestors': [{'id': 'D012817', 'term': 'Sign...","{'ancestors': [{'id': 'D000932', 'term': 'Anti...","{'conditions': [{'id': 'D009325', 'term': 'Nau..."
2,NCT00497666,NaT,Association Between Rosiglitazone Use and Clin...,Retrospective Study Evaluating the Association...,None,OBSERVATIONAL,UNKNOWN,2007-07-06,NaT,NaT,...,"[{'city': 'Zerifin,', 'country': 'Israel', 'fa...",None,None,None,"[{'affiliation': 'Nephrology Division, Assaf H...",None,"{""versionHolder"": ""2025-06-18""}","{'ancestors': [{'id': 'D014570', 'term': 'Urol...","{'ancestors': None, 'browseBranches': [{'abbre...","{'conditions': [{'id': 'D007674', 'term': 'Kid..."
3,NCT01978366,NaT,Open Label Extension Study of HT-100 in Patien...,An Open Label Extension Study of HT-100 in Pat...,None,INTERVENTIONAL,TERMINATED,2013-10-31,NaT,2016-04-30,...,"[{'city': 'Sacramento', 'country': 'United Sta...",None,None,"{'other': None, 'primary': [{'description': No...","[{'affiliation': 'AkashiTherapeutics', 'name':...",None,"{""versionHolder"": ""2025-06-18""}","{'ancestors': [{'id': 'D020966', 'term': 'Musc...","{'ancestors': [{'id': 'D000970', 'term': 'Anti...","{'conditions': [{'id': 'D009136', 'term': 'Mus..."
4,NCT01971866,NaT,Camera Based System to Monitor Patient Movemen...,Improving Safety and Accuracy of Proton Therap...,DS02,OBSERVATIONAL,TERMINATED,2013-10-17,NaT,NaT,...,"[{'city': 'Jacksonville', 'country': 'United S...",None,None,"{'other': None, 'primary': [{'description': No...",[{'affiliation': 'University of Florida Proton...,None,"{""versionHolder"": ""2025-06-18""}",None,None,None
5,NCT04912466,NaT,IBI322 Monotherapy or Combination Therapy in S...,"A Phase 1a/1b Study Evaluating the Safety, Tol...",None,INTERVENTIONAL,COMPLETED,2021-05-28,NaT,2022-12-03,...,"[{'city': 'Jinan', 'country': 'China', 'facili...",None,None,"{'other': None, 'primary': [{'description': No...","[{'affiliation': 'No.440, Jiyan Road, Jinang C...",None,"{""versionHolder"": ""2025-06-18""}","{'ancestors': None, 'browseBranches': None, 'b...","{'ancestors': None, 'browseBranches': [{'abbre...","{'conditions': [{'id': 'D009369', 'term': 'Neo..."
6,NCT03409666,NaT,Randomized Controlled RSA Study Comparing the ...,Randomized Controlled Study Comparing the Tape...,None,INTERVENTIONAL,COMPLETED,2018-01-08,NaT,2020-11-06,...,"[{'city': 'Roosendaal', 'country': 'Netherland...",None,None,"{'other': None, 'primary': [{'description': No...","[{'affiliation': 'Zimmer Biomet', 'name': 'Pao...",None,"{""versionHolder"": ""2025-06-18""}","{'ancestors': [{'id': 'D001168', 'term': 'Arth...",None,"{'conditions': [{'id': 'D010003', 'term': 'Ost..."
7,NCT00622466,NaT,Sorafenib and Paclitaxel in Treating Patients ...,Phase II Trial of Sorafenib and Paclitaxel for...,None,INTERVENTIONAL,TERMINATED,2008-02-22,NaT

## Features used for training

According to desctiption of the project we will use the following features from the dataset to carry out the task, the fetaures can be groupped into 4 parts, which are the following:

#### Free text:
 - `brief_title`
 - `brief_summary`
 - `detailed_description`
 - `eligibility_criteria`
 - `keywords`

#### Structured text data:
 - `mesh_terms`
 - `condition_browse_module`
 - `intervention_browse_module`

#### Embeddings of free text:
 - `brief_summary_embedding`
 - `eligibility_criteria_embedding`

#### Validation data for extracted topics:
 - `conditions`
 - `interventions`

 These features contain useful information related to medical text and context. The **"Free text"** and **"Structured text"** groups will give the backbone of the project's training data, and we can validate the topics using the features in the **"Validation data ..."** group. As for now the **"Embeddings .."** group is also listed here, we may use that later to speed up the processing pipeline or to combine it with our embeddings (this will be determined in Milestone 3. as we progress with the project and the training of the selected model, so for now it is just a nice-to-have, but we count on it if things go south later.)

In [66]:
FREE_TEXT_COLS = [
    "brief_title",
    "brief_summary",
    "detailed_description",
    "eligibility_criteria",
    "keywords",
]

STRUCTURED_TEXT_COLS = [
    "mesh_terms",
    "condition_browse_module",
    "intervention_browse_module",
]

VALIDATION_COL = ["conditions", "interventions"]

### Handling the data in batches

As the chosen dataset is remarkable in size, we choose to process it in batches to avoid problems realted to RAM.

In [67]:
def iterate_batches(ds, batch_size=2000):
    batch = []
    for row in ds:
        batch.append(row)
        if len(batch) == batch_size:
            yield batch
            batch = []
    if batch:
        yield batch